# Hello Blqs

In this notebook you will find a simple explanation for the motivation for blqs.  First make sure blqs and cirq are installed.

In [1]:
try:
    import blqs
except ImportError:
    print("installing blqs...")
    !pip install --quiet blqs
    print("installed blqs.")
    import blqs

try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")
    import cirq

This is an example of how to create a circuit using Cirq.  One sees that one has a base object upon which you append the elements.
Qisket and other frameworks follow a similar pattern.

In [2]:
q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit()
circuit.append(cirq.H(q0))
circuit.append([cirq.H(q1), cirq.CX(q0, q1)])
print(circuit)

0: ───H───@───
          │
1: ───H───X───


Suppose you would like to do something like
```python
H(0)
H(1)
for i in range(100):
   H(0)
   H(1)
   CX(0, 1)
```
Then you do something like this

In [3]:
multi_circuit = cirq.Circuit(
    cirq.H(q0), cirq.H(q1), cirq.CircuitOperation(circuit=circuit.freeze(), repetitions=100)
)
print(multi_circuit)

          Circuit_0x8c624a579ce341b9:
0: ───H───[ 0: ───H───@───          ]──────────────
          [           │             ]
          [ 1: ───H───X───          ](loops=100)
          │
1: ───H───#2───────────────────────────────────────


Wouldn't it be nice to be able to do this in idiomatic Python instead of with that code?
This is the idea behind blqs:
```
  BBB   L     QQ     SSSS
  B  B  L    Q  Q   S 
  BBB   L    Q  Q    SSS
  B  B  L    Q  Q       S
  BBB   LLLL  QQ Q  SSSS
```

In [4]:
import blqs

Let's just give a simple example

Define some simple ops. These could be quantum gates like is needed for Cirq above.
In general there are specific language you can use, or build your own
for what these are.  Here we just use these ops as a simple example.

In [5]:
H = blqs.Op("H")
CX = blqs.Op("CX")

# Now build a program out of these:
@blqs.build
def hello_blqs():
    H(0)
    H(1)


program = hello_blqs()
print(f"Program:\n{program}")

Program:
H 0
H 1


What is this program that is returned?

In [6]:
type(program)

blqs.program.Program

A program is just a sequence of program statements

In [7]:
for i, statement in enumerate(program):
    print(f"statement {i}: {statement}")

statement 0: H 0
statement 1: H 1


Notice that we have not had to add these to the program, they have been captured
in the decorated function.

Programs are just top level Blocks.  We will see blocks are a key component of blqs.

In [8]:
print(isinstance(program, blqs.Block))

True


Now lets build something a bit more interesting.

In [9]:
M = blqs.Op("M")


@blqs.build
def hello_conditional_blqs():
    H(0)
    M(0, blqs.Register("a"))
    if blqs.Register("a"):
        H(0)
    else:
        H(1)


program = hello_conditional_blqs()
print(f"Program:\n{program}")

Program:
H 0
M 0, R(a)
if R(a):
  H 0
else:
  H 1


Lets look at this program and its statements.

In [10]:
for s in program:
    print(type(s))

<class 'blqs.instruction.Instruction'>
<class 'blqs.instruction.Instruction'>
<class 'blqs.conditional.If'>


Notice that the third statement is an If object. Lets look at what this is:

In [11]:
if_statement = program[2]
print(f"condition: {if_statement.condition()}")
print(f"if block: {if_statement.if_block()}")
print(f"else block: {if_statement.else_block()}")

condition: R(a)
if block:   H 0
else block:   H 1


We see that we have written normal Python, but it has been translated over into 
an object that contains the Python objects we used in the code.
It has captured the condition, as well as the two possible paths for the if.
 
But what if we want to use a "normal" Python if?

In [12]:
@blqs.build
def hello_normal_if_blqs(a):
    H(0)
    if a:
        H(0)
    else:
        H(1)


program = hello_normal_if_blqs(True)
print(f"a=True program:\n{program}\n")
program = hello_normal_if_blqs(False)
print(f"a=False program:\n{program}")

a=True program:
H 0
H 0

a=False program:
H 0
H 1


We see that we get a program that executed Python and generated the appropriate program.  We can even mix and match.

In [13]:
@blqs.build
def hello_mix_match(a):
    H(0)
    if a:
        M(
            0,
        )
        if blqs.Register("b"):
            H(1)
    else:
        H(1)
    CX(0, 1)


program = hello_mix_match(True)


def my_blah():
    def my_program():
        H(0)

    return blqs.build(my_program)()


print(f"Program:\n{program}")

Program:
H 0
M 0
if R(b):
  H 1
CX 0, 1


We can do this for other Python native constructs.

In [14]:
@blqs.build
def hello_for_loop():
    # We use a dummy iterable here.
    for x in blqs.Iterable("range(4)", blqs.Register("x")):
        H(0)
        b = blqs.Register("b")
        M(0, b)
        if b:
            H(0)
        else:
            H(1)


program = hello_for_loop()
print(f"Program:\n{program}")

Program:
for R(x) in range(4):
  H 0
  b = R(b)
  M 0, R(b)
  if R(b):
    H 0
  else:
    H 1


This could represents the classical controller beside the quantum computer
running a for loop over 'x' and then conditional on that value (assume 0=False)
we implement H on 0 or on 1.

Blqs is a framework written to build other frameworks upon. If you'd like to learn more about blqs, a good place to start is the [introduction to blqs](docs/intro.md).  For a concrete application of blqs to Cirq, see [blqs_cirq](../blqs_cirq).